In [ ]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow import keras
from sklearn import model_selection
from sklearn.metrics import classification_report
import imgaug.augmenters as iaa
import imgaug as ia

In [ ]:
#iamge augmenation
ia.seed(1)
def Data_Aug(samples):
    randomaug = iaa.Sometimes(
        0.5,
        iaa.OneOf([
            iaa.Sequential([
                iaa.AllChannelsCLAHE(clip_limit=(1, 10)),
                iaa.Affine(rotate=(-5, 5))
            ]),
            iaa.Sequential([
                iaa.Affine(translate_percent={
                    "x": (-0.2, 0.2),
                    "y": (-0.2, 0.2)
                }),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.Affine(rotate=(-10, 10)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.PerspectiveTransform(scale=(0, 0.15)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ])
        ]))


    return randomaug.augment_images(samples)

def resize_images(imgs, hight, width):
    seq = iaa.Resize({"height": hight, "width": width})
    aug = seq.augment_images(imgs)
    return aug

In [ ]:
class cifar10data(keras.utils.Sequence):
    '''
    generate data batch of cifar10
    10 classes
    '''
    def __init__(self, data_x, data_y, batch_size, train):
        self.data_x = data_x
        self.data_y = data_y
        self.train = train
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.data_y) / self.batch_size))

    def __getitem__(self, idx):

        batch_x = self.data_x[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        batch_y = self.data_y[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        if self.train == True:
            batch_x = Data_Aug(batch_x)

        batch_x = np.clip(batch_x / 255.0, 0, 1).astype("float32")
        batch_y = keras.utils.to_categorical(np.array(batch_y), 10)
        return batch_x, [batch_y, batch_y, batch_y, batch_y, batch_y]

In [ ]:
#callback for save model
class CustomEarlyStoppingAndSave(keras.callbacks.Callback):
    def __init__(self, save_path, patience=0):
        super(CustomEarlyStoppingAndSave, self).__init__()
        self.patience = patience
        self.save_path = save_path
        self.best_weights = None

    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        #self.best_val_distance_loss = np.Inf
        self.best_val_acc = 0

    def on_epoch_end(self, epoch, logs=None):
        #val_distance_loss = logs.get('val_MMD_loss')
        val_acc = logs.get('val_model_accuracy') + logs.get(
            'val_model_1_accuracy') + logs.get(
                'val_model_2_accuracy') + logs.get(
                    'val_model_3_accuracy') + logs.get('val_model_4_accuracy')

        if  np.greater(val_acc, self.best_val_acc):
            #self.best_val_distance_loss = val_distance_loss
            self.best_val_acc = val_acc
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
            self.stopped_epoch = epoch
        #else:
            #self.wait += 1
            #if self.wait >= self.patience:
                #self.stopped_epoch = epoch
                #self.model.stop_training = True
                #print("Restoring model weights from the end of the best epoch.")
                #self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        self.model.set_weights(self.best_weights)
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping and save the model" %
                  (self.stopped_epoch + 1))
        self.model.save(self.save_path)

In [ ]:
#Cosine&Euclidean distance loss
def descriptor(X, mask = False, max_norm = False):
  X = tf.reduce_mean(X, axis=3)
  if mask:
    mean_X = tf.reduce_mean(X, axis = [1,2], keepdims= True)
    X = tf.where(X > mean_X, X , tf.zeros_like(X, dtype=tf.float32))
  if max_norm:
    max_X=tf.reduce_max(X, axis = [1,2], keepdims= True)
    X = tf.math.divide_no_nan(X, max_X)
  return X 
  
def compute_euclidean_sum(X, Y):
  L2_distance = tf.reduce_mean(tf.exp(-(X - Y)**2), axis= [1,2])
  return tf.reduce_mean(L2_distance)

def compute_cosine_sum(X, Y):
  X = tf.reshape(X, [tf.shape(X)[0], -1])
  Y = tf.reshape(Y, [tf.shape(Y)[0], -1])
  loss = -keras.losses.cosine_similarity(X,Y,axis=-1)
  return tf.reduce_mean(loss)

def cosine_euclidean_sum_loss(x, cosine_weight, euclidean_weight, mask, max_norm):
  descriptor0 = descriptor(x[0], mask=mask, max_norm=max_norm)
  descriptor1 = descriptor(x[1], mask=mask, max_norm=max_norm)
  descriptor2 = descriptor(x[2], mask=mask, max_norm=max_norm)
  descriptor3 = descriptor(x[3], mask=mask, max_norm=max_norm)
  descriptor4 = descriptor(x[4], mask=mask, max_norm=max_norm)
  cosine_sum = compute_cosine_sum(descriptor0, descriptor1)+compute_cosine_sum(descriptor0, descriptor2)+compute_cosine_sum(descriptor0, descriptor3)+compute_cosine_sum(descriptor0, descriptor4)+compute_cosine_sum(descriptor1, descriptor2)+compute_cosine_sum(descriptor1, descriptor3)+compute_cosine_sum(descriptor1, descriptor4)+compute_cosine_sum(descriptor2, descriptor3)+compute_cosine_sum(descriptor2, descriptor4)+compute_cosine_sum(descriptor3, descriptor4)
  euclidean_sum = compute_euclidean_sum(descriptor0, descriptor1)+compute_euclidean_sum(descriptor0, descriptor2)+compute_euclidean_sum(descriptor0, descriptor3)+compute_euclidean_sum(descriptor0, descriptor4)+compute_euclidean_sum(descriptor1, descriptor2)+compute_euclidean_sum(descriptor1, descriptor3)+compute_euclidean_sum(descriptor1, descriptor4)+compute_euclidean_sum(descriptor2, descriptor3)+compute_euclidean_sum(descriptor2, descriptor4)+compute_euclidean_sum(descriptor3, descriptor4)
  return cosine_sum, euclidean_sum, cosine_weight*cosine_sum+euclidean_weight*euclidean_sum  


In [ ]:
#SSIM distance loss
def compute_ssim(X,Y):
    X = tf.expand_dims(X,axis=3)
    Y = tf.expand_dims(Y,axis=3)
    return tf.reduce_mean(tf.image.ssim(X,Y,max_val=1,filter_size=2))
def ssim_sum_loss(x, weight, mask, max_norm):
  descriptor0 = descriptor(x[0], mask=mask, max_norm=max_norm)
  descriptor1 = descriptor(x[1], mask=mask, max_norm=max_norm)
  descriptor2 = descriptor(x[2], mask=mask, max_norm=max_norm)
  descriptor3 = descriptor(x[3], mask=mask, max_norm=max_norm)
  descriptor4 = descriptor(x[4], mask=mask, max_norm=max_norm)

  ssim_sum = compute_ssim(descriptor0, descriptor1)+compute_ssim(descriptor0, descriptor2)+compute_ssim(descriptor0, descriptor3)+compute_ssim(descriptor0, descriptor4)+compute_ssim(descriptor1, descriptor2)+compute_ssim(descriptor1, descriptor3)+compute_ssim(descriptor1, descriptor4)+compute_ssim(descriptor2, descriptor3)+compute_ssim(descriptor2, descriptor4)+compute_ssim(descriptor3, descriptor4)
  return ssim_sum, weight*ssim_sum  

In [ ]:
#create single base VGG architecture
def create_1_model(input_size, random_seed, numclasses):
  inputs = keras.Input(shape=input_size)
  x = keras.layers.Conv2D(64, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Conv2D(64, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  p1 = keras.layers.MaxPooling2D((2, 2))(x)
  x = keras.layers.Dropout(0.2)(p1)

  x = keras.layers.Conv2D(128, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Conv2D(128, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  p2 = keras.layers.MaxPooling2D((2, 2))(x)
  x = keras.layers.Dropout(0.3)(p2)

  x = keras.layers.Conv2D(256, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  f6 = keras.layers.Conv2D(256, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(f6)
  r3 = keras.layers.Activation('relu')(x)
  p3 = keras.layers.MaxPooling2D((2, 2))(r3)
  x = keras.layers.Dropout(0.4)(p3)

  x = keras.layers.Flatten()(x)
  x = keras.layers.Dense(128,  kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Dropout(0.5)(x)
  outputs = keras.layers.Dense(numclasses, activation='softmax', kernel_initializer=keras.initializers.HeUniform(seed=random_seed))(x)
  return keras.Model(inputs=inputs, outputs=[outputs, p3])

In [ ]:
#create five base models for training 
input_size = (32, 32, 3)

model0 = create_1_model(input_size=input_size,
                             random_seed=1,
                             numclasses=10)
model1 = create_1_model(input_size=input_size,
                             random_seed=2,
                             numclasses=10)
model2 = create_1_model(input_size=input_size,
                             random_seed=3,
                             numclasses=10)
model3 = create_1_model(input_size=input_size,
                             random_seed=4,
                             numclasses=10)
model4 = create_1_model(input_size=input_size,
                             random_seed=5,
                             numclasses=10)
allinputs = keras.Input(shape=input_size)
output0, feature_maps0 = model0(allinputs)
output1, feature_maps1 = model1(allinputs)
output2, feature_maps2 = model2(allinputs)
output3, feature_maps3 = model3(allinputs)
output4, feature_maps4 = model4(allinputs)
model_train = keras.Model(
    inputs=allinputs, outputs=[output0, output1, output2, output3, output4])

#implement distance loss
loss1, loss2, loss = cosine_euclidean_sum_loss(x=[feature_maps0, feature_maps1, feature_maps2, feature_maps3, feature_maps4], 
                                      cosine_weight=1, euclidean_weight=10, mask=True, max_norm=False)

model_train.add_loss(loss)
model_train.add_metric(loss1, name="loss1", aggregation='mean')
model_train.add_metric(loss2, name="loss2", aggregation='mean')

model_train.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                    loss=['categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy'],
                    metrics=['accuracy'])

#load training and validation data
(x_origin_train, y_origin_train), (x_test,y_test) = keras.datasets.cifar10.load_data()

y_origin_train = y_origin_train.reshape(-1,)
y_test = y_test.reshape(-1,)
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x_origin_train,
    y_origin_train,
    test_size=0.25,
    random_state=0,
    stratify=y_origin_train)
_, x_val, _, y_val = model_selection.train_test_split(
    x_val,
    y_val,
    test_size=2000,
    random_state=0,
    stratify=y_val)
#randomly select training samples
train_index = []
random.seed(0)
for i in range(10):
    train_index += (random.sample(list(np.where(y_train == i)[0]), 400)) # number of training samples per class
random.shuffle(train_index)
sampled_x_train = x_train[train_index]
sampled_y_train = y_train[train_index]
del train_index

train_sequence = cifar10data(sampled_x_train, sampled_y_train, batch_size=10, train=True)
validation_sequence = cifar10data(x_val, y_val, batch_size=10, train=False)

#train base models and save parameters
history = model_train.fit(
    train_sequence,
    epochs=300,
    validation_data=validation_sequence,
    callbacks=CustomEarlyStoppingAndSave(
        save_path='100_samples_seed_fix.h5',
        patience=10))
frame = pd.DataFrame({
    'loss':
    history.history['loss'],
    'loss1':
    history.history['loss1'],
    'loss2':
    history.history['loss2'],
    'model_0_loss':
    history.history['model_loss'],
    'model_1_loss':
    history.history['model_1_loss'],
    'model_2_loss':
    history.history['model_2_loss'],
    'model_3_loss':
    history.history['model_3_loss'],
    'model_4_loss':
    history.history['model_4_loss'],
    'model_0_accuracy':
    history.history['model_accuracy'],
    'model_1_accuracy':
    history.history['model_1_accuracy'],
    'model_2_accuracy':
    history.history['model_2_accuracy'],
    'model_3_accuracy':
    history.history['model_3_accuracy'],
    'model_4_accuracy':
    history.history['model_4_accuracy'],
    'val_loss':
    history.history['val_loss'],
    'val_loss1':
    history.history['val_loss1'],
    'val_loss2':
    history.history['val_loss2'],
    'val_model_0_loss':
    history.history['val_model_loss'],
    'val_model_1_loss':
    history.history['val_model_1_loss'],
    'val_model_2_loss':
    history.history['val_model_2_loss'],
    'val_model_3_loss':
    history.history['val_model_3_loss'],
    'val_model_4_loss':
    history.history['val_model_4_loss'],
    'val_model_0_accuracy':
    history.history['val_model_accuracy'],
    'val_model_1_accuracy':
    history.history['val_model_1_accuracy'],
    'val_model_2_accuracy':
    history.history['val_model_2_accuracy'],
    'val_model_3_accuracy':
    history.history['val_model_3_accuracy'],
    'val_model_4_accuracy':
    history.history['val_model_4_accuracy'],
})
frame.to_excel('table_100_samples_seed_fix.xlsx')


170508288/170498071 [==============================] - 6s 0us/step
Epoch 1/300
400/400 [==============================] - 56s 67ms/step - loss: 55.6445 - model_loss: 2.4642 - model_1_loss: 2.5055 - model_2_loss: 2.4738 - model_3_loss: 2.4673 - model_4_loss: 2.4497 - model_accuracy: 0.1910 - model_1_accuracy: 0.1805 - model_2_accuracy: 0.1770 - model_3_accuracy: 0.1865 - model_4_accuracy: 0.1890 - loss1: 2.2951 - loss2: 4.0989 - val_loss: 48.9119 - val_model_loss: 1.8928 - val_model_1_loss: 2.7053 - val_model_2_loss: 2.0427 - val_model_3_loss: 2.7384 - val_model_4_loss: 1.7451 - val_model_accuracy: 0.2915 - val_model_1_accuracy: 0.2435 - val_model_2_accuracy: 0.2725 - val_model_3_accuracy: 0.1990 - val_model_4_accuracy: 0.3670 - val_loss1: 2.1861 - val_loss2: 3.5601
Epoch 2/300
400/400 [==============================] - 25s 62ms/step - loss: 46.1917 - model_loss: 2.0353 - model_1_loss: 2.1376 - model_2_loss: 2.0489 - model_3_loss: 2.0377 - model_4_loss: 2.0549 - model_accuracy: 0.2645 -

In [ ]:
#evualate the five base models
x_test = x_test/255.0
evaluate = model_train.evaluate(x_test,[keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test)],batch_size=10)
evaluate


1000/1000 [==============================] - 13s 11ms/step - loss: 63.6597 - model_loss: 1.0181 - model_1_loss: 1.0228 - model_2_loss: 0.9945 - model_3_loss: 0.9913 - model_4_loss: 1.0082 - model_accuracy: 0.7357 - model_1_accuracy: 0.7440 - model_2_accuracy: 0.7481 - model_3_accuracy: 0.7450 - model_4_accuracy: 0.7284 - loss1: 0.0773 - loss2: 5.8548


In [ ]:
#hard voting ensemble
class HardVotingEnsemble:
    def __init__(self, model, testdata ,batch_size):
        self.ensemble_model = model
        self.testdata = testdata
        self.batch_size = batch_size
        self.data_size = testdata[1].shape[0]
    def prediction(self):
        predictions =[]
        for i in range(int(self.data_size/self.batch_size)):
            batch_predictions = np.argmax(self.ensemble_model.predict(self.testdata[0][i*self.batch_size:(i+1)*self.batch_size]),axis=2) 
            batch_predictions = np.stack(batch_predictions, axis=1)
            batch_predictions = [np.argmax(np.bincount(pre)) for pre in batch_predictions]
            predictions = predictions+batch_predictions
        return np.array(predictions)

    def evaluate(self):
        predictions = self.prediction()
        accuracy = np.sum(
            predictions == self.testdata[1]) / self.data_size
        return accuracy
HEnsemble = HardVotingEnsemble(model = model_train,testdata = (x_test, y_test),batch_size=10)
HEnsemble.evaluate()

0.7759

In [ ]:
#soft voting ensemble
def SoftVotingEnsemble(model, x, y):
  data_size = x.shape[0]
  predictions = tf.argmax(tf.reduce_mean(model.predict(x), axis = 0),axis=1)
  accuracy = np.sum(
     predictions == y) / data_size
  return accuracy
SoftVotingEnsemble(model_train, x_test, y_test)

0.7822

In [ ]:
#callback for ensemble learning
class eCustomEarlyStoppingAndSave(keras.callbacks.Callback):
    def __init__(self, save_path, patience=0):
        super(eCustomEarlyStoppingAndSave, self).__init__()
        self.patience = patience
        self.save_path = save_path
        self.best_weights = None

    def on_train_begin(self, logs=None):
        #self.wait = 0
        self.stopped_epoch = 0
        #self.best_val_distance_loss = np.Inf
        self.best_val_acc = 0

    def on_epoch_end(self, epoch, logs=None):
        #val_distance_loss = logs.get('val_MMD_loss')
        val_acc = logs.get('val_accuracy') 

        if  np.greater(val_acc, self.best_val_acc):
            #self.best_val_distance_loss = val_distance_loss
            self.best_val_acc = val_acc
            #self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
            self.stopped_epoch = epoch
        #else:
            #self.wait += 1
            #if self.wait >= self.patience:
                #self.stopped_epoch = epoch
                #self.model.stop_training = True
                #print("Restoring model weights from the end of the best epoch.")
                #self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        self.model.set_weights(self.best_weights)
        if self.stopped_epoch >= 0:
            print("Epoch %05d: early stopping and save the model" %
                  (self.stopped_epoch + 1))
        #self.model.save(self.save_path)

In [ ]:
#samples for ensemble learning 
class ecifar10data(keras.utils.Sequence):
    '''
    generate data batch of cifar10
    10 classes
    '''
    def __init__(self, data_x, data_y, batch_size, train):
        self.data_x = data_x
        self.data_y = data_y
        self.train = train
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.data_y) / self.batch_size))

    def __getitem__(self, idx):

        batch_x = self.data_x[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        batch_y = self.data_y[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        if self.train == True:
            batch_x = Data_Aug(batch_x)

        batch_x = np.clip(batch_x / 255.0, 0, 1).astype("float32")
        batch_y = keras.utils.to_categorical(np.array(batch_y), 10)
        return batch_x, batch_y

In [ ]:
#feature fusion model
model_path = '100_samples_seed_fix.h5'
model = keras.models.load_model(model_path)
extractor_1 = keras.models.Model(inputs=model.layers[1].inputs,outputs=model.layers[1].layers[24].output)
extractor_2 = keras.models.Model(inputs=model.layers[2].inputs,outputs=model.layers[2].layers[24].output)
extractor_3 = keras.models.Model(inputs=model.layers[3].inputs,outputs=model.layers[3].layers[24].output)
extractor_4 = keras.models.Model(inputs=model.layers[4].inputs,outputs=model.layers[4].layers[24].output)
extractor_5 = keras.models.Model(inputs=model.layers[5].inputs,outputs=model.layers[5].layers[24].output)
extractor_1.trainable = False
extractor_2.trainable = False
extractor_3.trainable = False
extractor_4.trainable = False
extractor_5.trainable = False
inputs = keras.Input(shape = input_size)
features_1 = extractor_1(inputs, training = False)
features_2 = extractor_2(inputs, training = False)
features_3 = extractor_3(inputs, training = False)
features_4 = extractor_4(inputs, training = False)
features_5 = extractor_5(inputs, training = False)

#avg_1 = keras.layers.GlobalAvgPool2D()(features_1)
#max_1 = keras.layers.GlobalMaxPool2D()(features_1)
#avg_2 = keras.layers.GlobalAvgPool2D()(features_2)
#max_2 = keras.layers.GlobalMaxPool2D()(features_2)
#avg_3 = keras.layers.GlobalAvgPool2D()(features_3)
#max_3 = keras.layers.GlobalMaxPool2D()(features_3)
#avg_4 = keras.layers.GlobalAvgPool2D()(features_4)
#max_4 = keras.layers.GlobalMaxPool2D()(features_4)
#avg_5 = keras.layers.GlobalAvgPool2D()(features_5)
#max_5 = keras.layers.GlobalMaxPool2D()(features_5)
#avg_1 = keras.layers.Lambda(lambda x: tf.math.l2_normalize(x,axis=1))(avg_1)
#max_1 = keras.layers.Lambda(lambda x: tf.math.l2_normalize(x,axis=1))(max_1)
#avg_2 = keras.layers.Lambda(lambda x: tf.math.l2_normalize(x,axis=1))(avg_2)
#max_2 = keras.layers.Lambda(lambda x: tf.math.l2_normalize(x,axis=1))(max_2)
#avg_3 = keras.layers.Lambda(lambda x: tf.math.l2_normalize(x,axis=1))(avg_3)
#max_3 = keras.layers.Lambda(lambda x: tf.math.l2_normalize(x,axis=1))(max_3)
#avg_4 = keras.layers.Lambda(lambda x: tf.math.l2_normalize(x,axis=1))(avg_4)
#max_4 = keras.layers.Lambda(lambda x: tf.math.l2_normalize(x,axis=1))(max_4)
#avg_5 = keras.layers.Lambda(lambda x: tf.math.l2_normalize(x,axis=1))(avg_5)
#max_5 = keras.layers.Lambda(lambda x: tf.math.l2_normalize(x,axis=1))(max_5)
#x = keras.layers.Concatenate(axis=-1)([avg_1,max_1,avg_2,max_2,avg_3,max_3,avg_4,max_4,avg_5,max_5])

x = keras.layers.Concatenate(axis=-1)([features_1,features_2,features_3,features_4,features_5])
#x = keras.layers.add([features_1,features_2,features_3,features_4,features_5])
#x = keras.layers.SeparableConv2D(128, (3, 3), padding='same',  kernel_initializer=keras.initializers.HeUniform(seed=None))(x)
#x = keras.layers.BatchNormalization()(x)
#x = keras.layers.Activation('relu')(x)
#x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128,  name='dense_100')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(10, activation='softmax', name='dense_101')(x)
ensemble = keras.Model(inputs = inputs, outputs = outputs)
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
train_sequence = ecifar10data(sampled_x_train, sampled_y_train, batch_size=10, train=False)
validation_sequence = ecifar10data(x_val, y_val, batch_size=10, train=False)
history = ensemble.fit(
    train_sequence,
    epochs=50,
    validation_data=validation_sequence,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='{numsample}_samples_seed_{numseed}.h5'.format(
            numsample=100, numseed='fix'),
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/50
400/400 [==============================] - 9s 12ms/step - loss: 0.1507 - accuracy: 0.9722 - val_loss: 0.7204 - val_accuracy: 0.7725
Epoch 2/50
400/400 [==============================] - 4s 11ms/step - loss: 0.0192 - accuracy: 0.9992 - val_loss: 0.7142 - val_accuracy: 0.7935
Epoch 3/50
400/400 [==============================] - 4s 11ms/step - loss: 0.0100 - accuracy: 0.9998 - val_loss: 0.7640 - val_accuracy: 0.7935
Epoch 4/50
400/400 [==============================] - 4s 11ms/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 0.8014 - val_accuracy: 0.7890
Epoch 5/50
400/400 [==============================] - 4s 11ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.8153 - val_accuracy: 0.7910
Epoch 6/50
400/400 [==============================] - 4s 11ms/step - loss: 0.0045 - accuracy: 0.9998 - val_loss: 0.8525 - val_accuracy: 0.7890
Epoch 7/50
400/400 [==============================] - 4s 11ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.8641 - val_accuracy: 0.7940

In [ ]:
#fine tuning
extractor_1.trainable = True
extractor_2.trainable = True
extractor_3.trainable = True
extractor_4.trainable = True
extractor_5.trainable = True
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
history = ensemble.fit(
    train_sequence,
    epochs=20,
    validation_data=validation_sequence,
    #validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='{numsample}_samples_seed_{numseed}.h5'.format(
            numsample=100, numseed='fix'),
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/20
400/400 [==============================] - 13s 21ms/step - loss: 0.0016 - accuracy: 0.9998 - val_loss: 1.1586 - val_accuracy: 0.7955
Epoch 2/20
400/400 [==============================] - 8s 20ms/step - loss: 0.0019 - accuracy: 0.9998 - val_loss: 1.1632 - val_accuracy: 0.7935
Epoch 3/20
400/400 [==============================] - 8s 20ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 1.1590 - val_accuracy: 0.7940
Epoch 4/20
400/400 [==============================] - 8s 20ms/step - loss: 0.0018 - accuracy: 0.9992 - val_loss: 1.1648 - val_accuracy: 0.7945
Epoch 5/20
400/400 [==============================] - 8s 20ms/step - loss: 0.0018 - accuracy: 0.9995 - val_loss: 1.1665 - val_accuracy: 0.7940
Epoch 6/20
400/400 [==============================] - 8s 20ms/step - loss: 0.0015 - accuracy: 0.9995 - val_loss: 1.1528 - val_accuracy: 0.7935
Epoch 7/20
400/400 [==============================] - 8s 20ms/step - loss: 8.3940e-04 - accuracy: 1.0000 - val_loss: 1.1656 - val_accuracy: 0

In [ ]:
#f1 score
predict = tf.argmax(ensemble.predict(x_test),axis=-1)
classification_report(y_test, predict,digits=4)

'              precision    recall  f1-score   support\n\n           0     0.7717    0.8250    0.7975      1000\n           1     0.9059    0.9050    0.9055      1000\n           2     0.7140    0.6740    0.6934      1000\n           3     0.6536    0.6020    0.6268      1000\n           4     0.7136    0.7650    0.7384      1000\n           5     0.7422    0.6420    0.6885      1000\n           6     0.7993    0.8560    0.8267      1000\n           7     0.7892    0.8200    0.8043      1000\n           8     0.8613    0.8820    0.8715      1000\n           9     0.8715    0.8680    0.8697      1000\n\n    accuracy                         0.7839     10000\n   macro avg     0.7822    0.7839    0.7822     10000\nweighted avg     0.7822    0.7839    0.7822     10000\n'

In [ ]:
#stacking ensemble
model_path = '100_samples_seed_fix.h5'
model = keras.models.load_model(model_path)
model_1 = keras.models.Model(inputs=model.layers[1].inputs,outputs=model.layers[1].outputs[0])
model_2 = keras.models.Model(inputs=model.layers[2].inputs,outputs=model.layers[2].outputs[0])
model_3 = keras.models.Model(inputs=model.layers[3].inputs,outputs=model.layers[3].outputs[0])
model_4 = keras.models.Model(inputs=model.layers[4].inputs,outputs=model.layers[4].outputs[0])
model_5 = keras.models.Model(inputs=model.layers[5].inputs,outputs=model.layers[5].outputs[0])
model_1.trainable = False
model_2.trainable = False
model_3.trainable = False
model_4.trainable = False
model_5.trainable = False
inputs = keras.Input(shape = input_size)
prediction_1 = model_1(inputs, training = False)
prediction_2 = model_2(inputs, training = False)
prediction_3 = model_3(inputs, training = False)
prediction_4 = model_4(inputs, training = False)
prediction_5 = model_5(inputs, training = False)
x = keras.layers.Concatenate(axis=-1)([prediction_1,prediction_2,prediction_3,prediction_4,prediction_5])
x = keras.layers.Dense(128, activation='relu', name='dense_100')(x)
#x = keras.layers.BatchNormalization()(x)
#x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(10, activation='softmax', name='dense_101')(x)
ensemble = keras.Model(inputs = inputs, outputs = outputs)
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
history = ensemble.fit(
    train_sequence,
    epochs=50,
    validation_data=validation_sequence,
    validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='{numsample}_samples_seed_{numseed}.h5'.format(
            numsample=100, numseed='fix'),
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/50
400/400 [==============================] - 7s 13ms/step - loss: 0.2802 - accuracy: 0.9778 - val_loss: 0.8196 - val_accuracy: 0.7855
Epoch 2/50
400/400 [==============================] - 4s 11ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.9249 - val_accuracy: 0.7880
Epoch 3/50
400/400 [==============================] - 5s 11ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.9963 - val_accuracy: 0.7880
Epoch 4/50
400/400 [==============================] - 5s 11ms/step - loss: 5.1765e-04 - accuracy: 1.0000 - val_loss: 1.0516 - val_accuracy: 0.7890
Epoch 5/50
400/400 [==============================] - 4s 11ms/step - loss: 2.9557e-04 - accuracy: 1.0000 - val_loss: 1.0967 - val_accuracy: 0.7900
Epoch 6/50
400/400 [==============================] - 4s 11ms/step - loss: 1.8464e-04 - accuracy: 1.0000 - val_loss: 1.1366 - val_accuracy: 0.7895
Epoch 7/50
400/400 [==============================] - 5s 11ms/step - loss: 1.2260e-04 - accuracy: 1.0000 - val_loss: 1.1729 - val_

In [ ]:
#finetuning
model_1.trainable = True
model_2.trainable = True
model_3.trainable = True
model_4.trainable = True
model_5.trainable = True
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
history = ensemble.fit(
    train_sequence,
    epochs=20,
    validation_data=validation_sequence,
    validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='{numsample}_samples_seed_{numseed}.h5'.format(
            numsample=100, numseed='fix'),
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/20
400/400 [==============================] - 13s 24ms/step - loss: 1.8978e-04 - accuracy: 1.0000 - val_loss: 1.1315 - val_accuracy: 0.7875
Epoch 2/20
400/400 [==============================] - 9s 23ms/step - loss: 1.3651e-04 - accuracy: 1.0000 - val_loss: 1.1528 - val_accuracy: 0.7880
Epoch 3/20
400/400 [==============================] - 9s 23ms/step - loss: 1.0970e-04 - accuracy: 1.0000 - val_loss: 1.1735 - val_accuracy: 0.7890
Epoch 4/20
400/400 [==============================] - 9s 23ms/step - loss: 8.9219e-05 - accuracy: 1.0000 - val_loss: 1.1936 - val_accuracy: 0.7895
Epoch 5/20
400/400 [==============================] - 9s 22ms/step - loss: 7.3258e-05 - accuracy: 1.0000 - val_loss: 1.2129 - val_accuracy: 0.7900
Epoch 6/20
400/400 [==============================] - 9s 23ms/step - loss: 6.0566e-05 - accuracy: 1.0000 - val_loss: 1.2317 - val_accuracy: 0.7895
Epoch 7/20
400/400 [==============================] - 9s 22ms/step - loss: 5.0319e-05 - accuracy: 1.0000 - val_loss: 

In [ ]:
#fi score
predict = tf.argmax(ensemble.predict(x_test),axis=-1)
classification_report(y_test, predict,digits=4)

'              precision    recall  f1-score   support\n\n           0     0.8067    0.7970    0.8018      1000\n           1     0.9066    0.8930    0.8997      1000\n           2     0.6900    0.6990    0.6945      1000\n           3     0.6378    0.6040    0.6204      1000\n           4     0.7520    0.7610    0.7565      1000\n           5     0.7198    0.6860    0.7025      1000\n           6     0.8110    0.8580    0.8338      1000\n           7     0.7970    0.8090    0.8030      1000\n           8     0.8581    0.8650    0.8616      1000\n           9     0.8678    0.8860    0.8768      1000\n\n    accuracy                         0.7858     10000\n   macro avg     0.7847    0.7858    0.7851     10000\nweighted avg     0.7847    0.7858    0.7851     10000\n'